In [1]:
# To deal with 7_PAH_zf_morphology

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, time

In [2]:
from datetime import datetime as dt
today = dt.now()
time_now_date = today.strftime('%Y_%m_%d')

In [3]:
# old - used to work
#complete_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/7_PAH/01_11_2021/tall/7_PAH_zf_morphology_data_2021JAN11.csv'

# github input file 
complete_file_path = '/Users/kimd999/7_PAH_zf_morphology_data_2020NOV11_tall_3756.csv'

df_morph = pd.read_csv(complete_file_path, header = 0)

In [4]:
df_morph.head(2400).tail

<bound method NDFrame.tail of       chemical.id bottle.id  conc  plate.id well      date endpoint  value
0            3756   C761434   0.0     20544  H01  20200701     MO24    0.0
1            3756   C761434   0.0     20544  H02  20200701     MO24    1.0
2            3756   C761434   0.0     20544  H03  20200701     MO24    0.0
3            3756   C761434   0.0     20544  H04  20200701     MO24    0.0
4            3756   C761434   0.0     20544  H05  20200701     MO24    0.0
...           ...       ...   ...       ...  ...       ...      ...    ...
2395         3756   C761434   6.8     20624  F08  20200701     LTRK    0.0
2396         3756   C761434   6.8     20624  F09  20200701     LTRK    0.0
2397         3756   C761434   6.8     20624  F10  20200701     LTRK    0.0
2398         3756   C761434   6.8     20624  F11  20200701     LTRK    0.0
2399         3756   C761434   6.8     20624  F12  20200701     LTRK    0.0

[2400 rows x 8 columns]>

In [22]:
#np.sum(morph_all_data['value'] == 1)

In [23]:
#np.sum(morph_all_data['value'] == 0) # -> 82%

In [24]:
#np.sum(np.isnan(morph_all_data['value']))

In [25]:
df_morph.shape

(28224, 8)

In [5]:
# Keep only relevant columns
columns_to_keep = ['chemical.id', 'conc', 'plate.id', 'well', 'endpoint', 'value']
df_morph_select = df_morph.loc[:,columns_to_keep]
df_morph_select.head()

,chemical.id,conc,plate.id,well,endpoint,value
0,3756,0.0,20544,H01,MO24,0.0
1,3756,0.0,20544,H02,MO24,1.0
2,3756,0.0,20544,H03,MO24,0.0
3,3756,0.0,20544,H04,MO24,0.0
4,3756,0.0,20544,H05,MO24,0.0


In [6]:
start_time = time.time()

df_reformatted = pd.DataFrame()
total_number_of_unique_chemicals = 0
total_number_of_chemical_plate_well = 0


full_devel = "full"
#full_devel = "devel"
    
if (full_devel == "full"):
    # all chemicals
    chemical_id_from_here = np.unique(df_morph['chemical.id'])
else: # full_devel = "devel"
    chemical_id_from_here = [3756]

for chemical_index in chemical_id_from_here:
    print("chemical_index:" + str(chemical_index))
    total_number_of_unique_chemicals += 1
    df_morph_chemical = df_morph_select.loc[df_morph['chemical.id'] == chemical_index,:]
    
    # Append chemical_plate_well as a unique identifier
    df_morph_chemical.insert(0, 'chemical_plate_well', df_morph_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))
    
    for cpw in np.unique(df_morph_chemical.chemical_plate_well):
        total_number_of_chemical_plate_well += 1
        temp_df = df_morph_chemical.loc[df_morph_chemical.chemical_plate_well == cpw,:]
        temp_df_grouped = temp_df.groupby(['chemical.id', 'plate.id', 'well'])
        for name, group in temp_df_grouped:
            try:
#            if(len(group.endpoint) == 14):
                temp = pd.DataFrame( {
                        'chemical.id': np.unique(temp_df['chemical.id']),
                        'plate.id': np.unique(temp_df['plate.id']),
                        'well': np.unique(temp_df['well']),
                        'chemical_plate_well': np.unique(temp_df['chemical_plate_well']),
                        'conc': np.unique(temp_df['conc']),
                        'AXIS': temp_df.value[temp_df.endpoint == 'AXIS'].values,
                        'BRN_': temp_df.value[temp_df.endpoint == 'BRN_'].values,
                        'CRAN': temp_df.value[temp_df.endpoint == 'CRAN'].values,
                        'DNC_': temp_df.value[temp_df.endpoint == 'DNC_'].values,
                        'DP24': temp_df.value[temp_df.endpoint == 'DP24'].values,
                        'EDEM': temp_df.value[temp_df.endpoint == 'EDEM'].values,
                        'LTRK': temp_df.value[temp_df.endpoint == 'LTRK'].values,
                        'MO24': temp_df.value[temp_df.endpoint == 'MO24'].values,
                        'MORT': temp_df.value[temp_df.endpoint == 'MORT'].values,
                        'MUSC': temp_df.value[temp_df.endpoint == 'MUSC'].values,
                        'NC__': temp_df.value[temp_df.endpoint == 'NC__'].values,
                        'SKIN': temp_df.value[temp_df.endpoint == 'SKIN'].values,
                        'SM24': temp_df.value[temp_df.endpoint == 'SM24'].values,
                        'TCHR': temp_df.value[temp_df.endpoint == 'TCHR'].values,  }  )
                df_reformatted = pd.concat([df_reformatted, temp])
            except:
                print ("len(group.endpoint) != 14")
                print ("chemical_plate_well:" + str(cpw))
print ("total_number_of_unique_chemicals:" + str(total_number_of_unique_chemicals))
print ("total_number_of_chemical_plate_well:" + str(total_number_of_chemical_plate_well))
end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("Reformatting is done, it took:"+str(time_took))
# For all 7 chemicals, it took 15 seconds

chemical_index:3756
total_number_of_unique_chemicals:1
total_number_of_chemical_plate_well:288
Reformatting is done, it took:2.1 seconds


In [7]:
pd.set_option('display.max_columns', None)
print (df_reformatted.head())
print ("df_reformatted.shape:" + str(df_reformatted.shape))

   chemical.id  plate.id well chemical_plate_well   conc  AXIS  BRN_  CRAN  \
0         3756     20544  A01      3756_20544_A01  100.0   NaN   NaN   NaN   
0         3756     20544  A02      3756_20544_A02  100.0   0.0   0.0   0.0   
0         3756     20544  A03      3756_20544_A03  100.0   NaN   NaN   NaN   
0         3756     20544  A04      3756_20544_A04  100.0   0.0   0.0   0.0   
0         3756     20544  A05      3756_20544_A05  100.0   0.0   0.0   0.0   

   DNC_  DP24  EDEM  LTRK  MO24  MORT  MUSC  NC__  SKIN  SM24  TCHR  
0   0.0   NaN   NaN   NaN   1.0   NaN   NaN   NaN   NaN   NaN   NaN  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
0   0.0   NaN   NaN   NaN   1.0   NaN   NaN   NaN   NaN   NaN   NaN  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
df_reformatted.shape:(288, 19)


In [8]:
output_file_name = complete_file_path[:-4] + "_wide_made_in_" + str(time_now_date) + '.csv'
print (f"output_file_name:{output_file_name}")

df_reformatted.to_csv(output_file_name,index=False)

output_file_name:/Users/kimd999/7_PAH_zf_morphology_data_2020NOV11_tall_3756_wide_made_in_2021_07_19.csv


In [30]:
a=b

NameError: name 'b' is not defined

## below is investigational

In [ ]:
reformat_data_DNC_0 = pd.DataFrame()

In [ ]:
reformat_data_DNC_0 = reformat_data.loc[reformat_data['DNC_'] == 0.0]
print ("reformat_data_DNC_0.shape:" + str(reformat_data_DNC_0.shape))

reformat_data_DNC_0.to_csv(output_file_name,index=False)

In [ ]:
'''
nan_count = 0
zero_count = 0
one_count = 0
for (columnName, columnData) in reformat_data.iteritems(): 
    if (columnName == "chemical.id") or (columnName == "plate.id") or (columnName == "well") or (columnName == "chemical_plate_well") or (columnName == "conc"):
        continue
#    print('Colunm Name : ', columnName) 
#    print('Column Contents : ', columnData.values[i])
#    print (len(columnData.values))
    for i in range(len(columnData.values)):
        if (str(columnData.values[i]) == "nan"):
            nan_count += 1
        elif (str(columnData.values[i]) == "0.0"):
            zero_count += 1
        elif (str(columnData.values[i]) == "1.0"):
            one_count += 1
#        print('Column Contents : ', columnData.values[i])
print ("nan_count:" + str(nan_count))
print ("zero_count:" + str(zero_count))
print ("one_count:" + str(one_count))
'''

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
len(np.unique(reformat_data['chemical.id']))